In [7]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import pandas as pd
import math

# Custom quantization function with STE
class QuantizeFunction(torch.autograd.Function):
    @staticmethod
    def forward(ctx, input, num_bits, signed=True):
        if signed:
            qmin = -2 ** (num_bits - 1)
            qmax = 2 ** (num_bits - 1) - 1
        else:
            qmin = 0
            qmax = 2 ** num_bits - 1
        max_val = (input.abs()).max()
        scale = -max_val / float(qmin)
        scale = scale + 1e-8
        input_q = torch.round(input / scale).clamp(qmin, qmax)
        return input_q * scale

    @staticmethod
    def backward(ctx, grad_output):
        return grad_output, None, None

def quantize_int4_ste(tensor):
    return QuantizeFunction.apply(tensor, 4, True)

def quantize_int32_ste(tensor):
    return QuantizeFunction.apply(tensor, 32, True)

# Custom quantized linear layer
class QuantLinear(nn.Module):
    def __init__(self, in_features, out_features):
        super(QuantLinear, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        # Initialize weights and biases
        self.weight = nn.Parameter(torch.Tensor(out_features, in_features))
        self.bias = nn.Parameter(torch.Tensor(out_features))
        # Initialize parameters
        self.weight_q = None
        self.bias_q = None
        self.reset_parameters()

    def reset_parameters(self):
        nn.init.kaiming_uniform_(self.weight, a=math.sqrt(5))
        fan_in, _ = nn.init._calculate_fan_in_and_fan_out(self.weight)
        bound = 1 / math.sqrt(fan_in)
        nn.init.uniform_(self.bias, -bound, bound)

    def forward(self, input):
        # Quantize weights and biases using STE
        weight_dq = quantize_int4_ste(self.weight)
        bias_dq = quantize_int4_ste(self.bias)

        self.weight_q = weight_dq.detach()
        self.bias_q = bias_dq.detach()
        # Quantize input using STE
        input_dq = quantize_int32_ste(input)
        # Compute output
        output = torch.matmul(input_dq, weight_dq.t()) + bias_dq
        return output

# Quantized model
class QuantComplexMLP(nn.Module):
    def __init__(self, input_size, hidden_size1, hidden_size2, hidden_size3, hidden_size4, hidden_size5, hidden_size6, hidden_size7,hidden_size8, output_size):
        super(QuantComplexMLP, self).__init__()
        # Define layers
        self.fc1 = QuantLinear(input_size, hidden_size1)
        self.relu1 = nn.ReLU()
        self.fc2 = QuantLinear(hidden_size1, hidden_size2)
        self.relu2 = nn.ReLU()
        self.fc3 = QuantLinear(hidden_size2, hidden_size3)
        self.relu3 = nn.ReLU()
        self.fc4 = QuantLinear(hidden_size3, hidden_size4)
        self.relu4 = nn.ReLU()
        self.fc5 = QuantLinear(hidden_size4, hidden_size5)
        self.relu5 = nn.ReLU()
        self.fc6 = QuantLinear(hidden_size5, hidden_size6)
        self.relu6 = nn.ReLU()
        self.fc7 = QuantLinear(hidden_size6, hidden_size7)
        self.relu7 = nn.ReLU()
        self.fc8 = QuantLinear(hidden_size7, hidden_size8)
        self.relu8 = nn.ReLU()
        self.fc9 = QuantLinear(hidden_size8, output_size)

    def forward(self, x):
        # Quantize input using STE
        x = quantize_int32_ste(x)
        x = self.fc1(x)
        x = self.relu1(x)
        x = self.fc2(x)
        x = self.relu2(x)
        x = self.fc3(x)
        x = self.relu3(x)
        x = self.fc4(x)
        x = self.relu4(x)
        x = self.fc5(x)
        x = self.relu5(x)
        x = self.fc6(x)
        x = self.relu6(x)
        x = self.fc7(x)
        x = self.relu7(x)
        x = self.fc8(x)
        x = self.relu8(x)
        x = self.fc9(x)
        # Quantize output to int16
        x = quantize_int32_ste(x)
        return x

# Device Configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Load Data
df = pd.read_csv('dataset1.csv')

input_size = 27   # Adjust based on your actual data
output_size = 6   # Adjust based on your actual data

train_data_columns = df.columns[:27]
train_label_columns = df.columns[27:]

train_data = torch.tensor(df[train_data_columns].values, dtype=torch.float32)
train_labels = torch.tensor(df[train_label_columns].values, dtype=torch.float32)

# Preprocess labels if needed
train_labels[:99] = train_labels[:99] * 99  # Adjust indices as needed

train_dataset = TensorDataset(train_data, train_labels)
train_loader = DataLoader(train_dataset, batch_size=1000, shuffle=True)

# Initialize the model
model = QuantComplexMLP(input_size, 512, 1024, 1024, 1024, 1024, 512, 512, 256, output_size).to(device)

# Define Loss and Optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.000008)

# Training loop
num_epochs = 3000  # Adjust as needed
for epoch in range(num_epochs):
    model.train()
    for batch_data, batch_labels in train_loader:
        batch_data = batch_data.to(device)
        batch_labels = batch_labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(batch_data)
        loss = criterion(outputs, batch_labels)
        loss.backward()
        optimizer.step()
    
    # Optional: Print loss every 100 epochs
    if (epoch + 1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')




Epoch [10/3000], Loss: 3202403328.0000
Epoch [20/3000], Loss: 3202092032.0000
Epoch [30/3000], Loss: 3201620736.0000
Epoch [40/3000], Loss: 3200798720.0000
Epoch [50/3000], Loss: 3199352576.0000
Epoch [60/3000], Loss: 3196859136.0000
Epoch [70/3000], Loss: 3192491264.0000
Epoch [80/3000], Loss: 3184631040.0000
Epoch [90/3000], Loss: 3170357248.0000
Epoch [100/3000], Loss: 3146097664.0000
Epoch [110/3000], Loss: 3103565568.0000
Epoch [120/3000], Loss: 3033294336.0000
Epoch [130/3000], Loss: 2920693248.0000
Epoch [140/3000], Loss: 2752096256.0000
Epoch [150/3000], Loss: 2522329344.0000
Epoch [160/3000], Loss: 2288157952.0000
Epoch [170/3000], Loss: 2169354240.0000
Epoch [180/3000], Loss: 2153435648.0000
Epoch [190/3000], Loss: 2124290688.0000
Epoch [200/3000], Loss: 2108579712.0000
Epoch [210/3000], Loss: 2098054400.0000
Epoch [220/3000], Loss: 2090438656.0000
Epoch [230/3000], Loss: 2084183040.0000
Epoch [240/3000], Loss: 2078262784.0000
Epoch [250/3000], Loss: 2072775168.0000
Epoch [26

In [8]:
test_data_columns = df.columns[:input_size]
# Preprocess the input data for inference
test_data = df[test_data_columns].values[:5] 

# Convert to tensor and send to device
test_data_tensor = torch.tensor(test_data, dtype=torch.float32).to(device)

model.eval()  

with torch.no_grad():
    outputs = model(test_data_tensor)

# Convert the outputs back to CPU and numpy for interpretation
predicted_outputs = outputs.cpu().numpy()

# Display the input and output
for i, (input_sample, output_sample) in enumerate(zip(test_data, predicted_outputs)):
    print(f"Input {i+1}: {input_sample}")
    
    print(f"Output {i+1}: {output_sample}")

Input 1: [ -3838   3208   4815  -7305   6274  10225 -26275  41177  27282   2089
   3387   4355   6191   3507   7220   4545 -11293   9543   2703   1994
   3383   3820  -3052   4308   2953  -4864   4688]
Output 1: [ 88568.34     7436.9404  42360.223  -24915.695  -39520.516  -92890.445 ]
Input 2: [ 8084 -8388  7723  2375  4568 -8298 -4767  2683 -5010  6827  7562  9324
  5928 -6200 -6543 -3901 -4269 -4617  4548  5250  4074  4703 -3112 -2981
  2071 -3698 -3190]
Output 2: [   917.91156  22456.293    21735.418   -82504.38     26362.166
  24566.473  ]
Input 3: [ -5811  -8151  -6229   7844  -5430  -5788   3645   5232  -2511 -13851
   8516 -12756 -15505  21240  18457  -7431   6370   4038   8048   5936
  -3913  -5424 -11431   7630  -6838  -5644   3774]
Output 3: [ 29460.697  46383.67   66453.02  -71872.13  -67268.86   17244.98 ]
Input 4: [  3319  -6346  -5967 -10336 -13020   3951   7558  -8733   6379  -6063
  -3525   2177  -7593   8790   8619   7475   6700   6663  -4378   3529
   2023  -3884   55

In [9]:
activations = {}

def get_activation(name):
    def hook(model, input, output):
        activations[name] = output.detach()
    return hook

# Register hooks on layers
model.fc1.register_forward_hook(get_activation('fc1'))
model.relu1.register_forward_hook(get_activation('relu1'))
model.fc2.register_forward_hook(get_activation('fc2'))
model.relu2.register_forward_hook(get_activation('relu2'))
model.fc3.register_forward_hook(get_activation('fc3'))
model.relu3.register_forward_hook(get_activation('relu3'))
model.fc4.register_forward_hook(get_activation('fc4'))
model.relu4.register_forward_hook(get_activation('relu4'))
model.fc5.register_forward_hook(get_activation('fc5'))
model.relu5.register_forward_hook(get_activation('relu5'))
model.fc6.register_forward_hook(get_activation('fc6'))
model.relu6.register_forward_hook(get_activation('relu6'))
model.fc7.register_forward_hook(get_activation('fc7'))
model.relu7.register_forward_hook(get_activation('relu7'))
model.fc8.register_forward_hook(get_activation('fc8'))
model.relu8.register_forward_hook(get_activation('relu8'))
model.fc9.register_forward_hook(get_activation('fc9'))

# Prepare test data
test_data_columns = df.columns[:input_size]
test_data = df[test_data_columns].values[:1]  # Select first 5 samples for inference
test_data_tensor = torch.tensor(test_data, dtype=torch.float32).to(device)

# Set model to evaluation mode
model.eval()

# Run inference
with torch.no_grad():
    outputs = model(test_data_tensor)

# Print quantized weights and biases
#print("Quantized Weights and Biases:")
#for name, module in model.named_modules():
 #   if isinstance(module, QuantLinear):
  #      print(f"\nLayer: {name}")
   #     print(f"Quantized Weight shape: {module.weight.shape}")
    #    print(module.weight_q)
     #   print(f"Quantized Bias shape: {module.bias.shape}")
      #  print(module.bias_q)

# Print activations
print("\nActivations:")
for name, activation in activations.items():
    print(f"\n{name} activation shape: {activation.shape}")
    print(activation)

# Print final outputs
predicted_outputs = outputs.cpu().numpy()
print("\nPredicted Outputs:")
print(predicted_outputs)


Activations:

fc1 activation shape: torch.Size([1, 512])
tensor([[ 1.7482e+04, -2.6551e+03, -8.6797e+03, -1.2320e+04,  9.4280e+03,
         -1.6745e+03, -3.0128e+03, -1.0744e+03,  1.0980e+04, -5.9156e+03,
          1.9357e+02, -1.6501e+03, -1.4498e+03, -6.5534e+02,  3.1754e+03,
         -9.3908e+03, -8.8080e+03, -5.2951e+03, -1.3023e+04,  1.4414e+03,
         -3.1708e+03, -8.6170e+03, -2.1148e+03,  8.9556e+03,  2.4073e+03,
          1.4141e+03,  1.5480e+03, -4.7046e+03,  6.6887e+03,  9.3681e+03,
          6.7036e+03, -1.0086e+02,  6.2549e+03,  1.3568e+04, -9.1081e+03,
         -9.3681e+03,  5.0824e+03, -2.5118e+03, -1.5317e+03,  9.2771e+03,
          1.0254e+04, -4.2481e+03, -1.2235e+04, -6.3425e+03,  1.3789e+03,
         -5.9843e+01,  8.8012e+03,  9.4499e+03,  1.9348e+03, -3.4037e+01,
          2.8812e+03,  1.3272e+04,  2.1764e+03,  7.8874e+03, -1.2153e+04,
         -6.6965e+03, -2.5547e+01,  6.4700e+03, -7.0378e+03, -6.6598e+03,
          6.8750e+03, -5.1995e+03, -1.0800e+04,  3.259

In [37]:
import struct
import os
def export_quantized_weights(model, output_dir='weights'):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    layer_idx = 0
    for name, module in model.named_modules():
        if isinstance(module, QuantLinear):
            qmin=-8
            qmax=7
            layer_idx += 1

            # Get weights and biases
            #weights = module.weight.detach().cpu()
            weights = module.weight_q
            biases = module.bias_q if module.bias is not None else None
            if(layer_idx==1) : print(weights[0])
            # Quantize weights
            
            max_val = max(weights.abs().max(),biases.abs().max())

            scale = -max_val / float(-8)
            quantized_weights= (torch.round(weights / scale).clamp(qmin, qmax))
            if(layer_idx==1) : print(quantized_weights[0])
          
            quantized_biases =(torch.round(biases / scale).clamp(qmin, qmax))

            # Convert quantized weights to uint8 for packing
            quantized_weights_uint = quantized_weights.to(torch.uint8)
            if(layer_idx==1) : print(quantized_weights_uint[0])
            lower_nibble = quantized_weights_uint & 0x0F
            
            # Flatten and pack weights
            flat_weights = lower_nibble.flatten()
            if flat_weights.numel() % 2 != 0:
                flat_weights = torch.cat([flat_weights, torch.tensor([0], dtype=torch.uint8)])
            packed_weights = (flat_weights[1::2] << 4) | flat_weights[0::2]
            packed_weights = packed_weights.numpy()

            # Save packed weights to a COE file
            weight_file = os.path.join(output_dir, f'fc{layer_idx}_weight.coe')
            with open(weight_file, 'w') as f:
                f.write('memory_initialization_radix=16;\n')
                f.write('memory_initialization_vector=\n')
                hex_values = [f'{byte:02X}' for byte in packed_weights]
                f.write(',\n'.join(hex_values))
                f.write(';')
            print(f"Saved quantized weights for layer {layer_idx} to {weight_file}")

            # Similarly handle biases if needed
            quantized_biases_uint = quantized_biases.to(torch.uint8)
    
            # Extract the lower 4 bits (4-bit two's complement representation)
            lower_nibble_bias = quantized_biases_uint & 0x0F
            
            # Flatten biases
            flat_biases = lower_nibble_bias.flatten()
            
            # Since we are storing one 4-bit bias per 8-bit byte, no packing is needed
            # The upper 4 bits are zero
            packed_biases = flat_biases.numpy()
            
            # Save biases to a COE file
            bias_file = os.path.join(output_dir, f'fc{layer_idx}_bias.coe')
            with open(bias_file, 'w') as f:
                f.write('memory_initialization_radix=16;\n')
                f.write('memory_initialization_vector=\n')
                hex_values = [f'{byte:02X}' for byte in packed_biases]
                f.write(',\n'.join(hex_values))
                f.write(';')
            print(f"Saved quantized biases for layer {layer_idx} to {bias_file}")

            # Optionally, save scale and zero_point for dequantization on FPGA
            params_file = os.path.join(output_dir, f'fc{layer_idx}_params.txt')
            with open(params_file, 'w') as f:
                f.write(f'scale_w: {scale}\n')
                #f.write(f'zero_point_w: {zero_point_w}\n')
              
            print(f"Saved quantization parameters for layer {layer_idx} to {params_file}")

In [38]:
def process_coe_file(input_filename, output_filename):
    with open(input_filename, 'r') as infile:
        lines = infile.readlines()

    # Initialize variables
    header_lines = []
    data_lines = []
    in_data_section = False

    # Process lines
    for line in lines:
        line = line.strip()
        if line.startswith('memory_initialization_vector'):
            in_data_section = True
            header_lines.append(line)
        elif in_data_section:
            # Remove any semicolons at the end
            line = line.rstrip(';')
            data_lines.append(line)
        else:
            header_lines.append(line)

    # Combine all data entries into a single string and split by commas
    data_str = ''.join(data_lines)
    data_entries = data_str.split(',')

    # Clean up the data entries
    data_entries = [entry.strip() for entry in data_entries if entry.strip()]

    # Now, each data entry contains two 4-bit numbers stored together
    # We need to split each data entry into two numbers
    numbers = []
    for entry in data_entries:
        # Assume entries are in hex format
        # Each entry is 2 hex digits (8 bits)
        # Split into higher and lower 4-bit numbers
        entry = entry.zfill(2)  # Ensure each entry has at least 2 hex digits
        if len(entry) != 2:
            print(f"Warning: data entry '{entry}' does not have 2 hex digits")
            continue
        high_digit = entry[0]  # Higher bits (second number)
        low_digit = entry[1]   # Lower bits (first number)
        # Append the numbers in the correct order
        numbers.append(low_digit)   # First number (lower bits)
        numbers.append(high_digit)  # Second number (higher bits)

    # Insert '0' after every 27 numbers
    processed_numbers = []
    for i, number in enumerate(numbers):
        processed_numbers.append(number)
        if (i + 1) % 27 == 0:
            processed_numbers.append('0')

    # Now, recombine the numbers back into data entries (second number + first number)
    processed_data_entries = []
    i = 0
    while i < len(processed_numbers):
        first_number = processed_numbers[i]
        if i + 1 < len(processed_numbers):
            second_number = processed_numbers[i+1]
            data_entry = second_number + first_number  # Second number (higher bits) + First number (lower bits)
            processed_data_entries.append(data_entry)
            i += 2
        else:
            # Only one number left, pad with zero
            data_entry = '0' + first_number  # Pad higher bits with zero
            processed_data_entries.append(data_entry)
            i += 1

    # Write the output file
    with open(output_filename, 'w') as outfile:
        for line in header_lines:
            outfile.write(line + '\n')
        outfile.write('\n')
        # Write the data entries with commas and end with a semicolon
        for i, entry in enumerate(processed_data_entries):
            if i != len(processed_data_entries) - 1:
                outfile.write(entry + ',\n')
            else:
                outfile.write(entry + ';\n')


In [40]:
export_quantized_weights(model)


tensor([-0.1587, -0.0793,  0.1587, -0.1058,  0.0529,  0.0264, -0.0529,  0.1587,
         0.1322,  0.1851,  0.1322,  0.1322, -0.1322,  0.0529,  0.0529, -0.0529,
        -0.0793,  0.1058,  0.0793,  0.1587, -0.0793,  0.0529,  0.0529,  0.1587,
        -0.0000, -0.1058, -0.1851])
tensor([-6., -3.,  6., -4.,  2.,  1., -2.,  6.,  5.,  7.,  5.,  5., -5.,  2.,
         2., -2., -3.,  4.,  3.,  6., -3.,  2.,  2.,  6., -0., -4., -7.])
tensor([250, 253,   6, 252,   2,   1, 254,   6,   5,   7,   5,   5, 251,   2,
          2, 254, 253,   4,   3,   6, 253,   2,   2,   6,   0, 252, 249],
       dtype=torch.uint8)
Saved quantized weights for layer 1 to weights\fc1_weight.coe
Saved quantized biases for layer 1 to weights\fc1_bias.coe
Saved quantization parameters for layer 1 to weights\fc1_params.txt
Saved quantized weights for layer 2 to weights\fc2_weight.coe
Saved quantized biases for layer 2 to weights\fc2_bias.coe
Saved quantization parameters for layer 2 to weights\fc2_params.txt
Saved quantized 

In [41]:
input_file = 'weights/fc1_weight.coe'
output_file = 'weights/fc1_weight_padding.coe'
process_coe_file(input_file, output_file)

In [45]:
valid_scale_output_on_FPGA = [x * (88568.34 / 45994) for x in [45994, 3980, 22019, -12232, -20958, -47683]]   
print(valid_scale_output_on_FPGA)

[88568.34, 7664.08647214854, 42400.88442970822, -23554.549177718833, -40357.7699204244, -91820.76262599468]
